In [1]:
# %matplotlib inline


# 1) Wipe out your namespace
%reset -f

# 2) Clear Jupyter’s stored outputs (and inputs if you like)
try:
    Out.clear()
except NameError:
    pass

try:
    In.clear()
except NameError:
    pass

# 3) Force Python GC
import gc
gc.collect()

# 4) Free any GPU buffers
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()


import importlib
from libs import params, trades, feats, plots, models_core, models_custom
importlib.reload(params)
importlib.reload(trades)
importlib.reload(feats)
importlib.reload(plots)
importlib.reload(models_core)
importlib.reload(models_custom)

<module 'libs.models_custom' from '/workspace/my_models/Trading/_Stock_Analysis_/libs/models_custom.py'>

In [2]:
# Turn off interactive plotting globally (we’ll manage our own display)
import matplotlib
import matplotlib.pyplot as plt
plt.ioff()

import pandas as pd
import numpy as np
import math

from pathlib import Path
import pickle
import datetime as dt
from datetime import datetime
from datetime import time

import seaborn as sns
from pprint import pprint

import torch.nn.functional as Funct
from torch.utils.data import Dataset, DataLoader
torch.serialization.add_safe_globals([models_core.DayWindowDataset])

from tqdm.auto import tqdm

import io
import os
import json
from PIL import Image
import IPython.display as disp

import optuna
from optuna.trial import TrialState
from optuna.importance import get_param_importances
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.storages import RDBStorage

In [3]:
df_trainval = pd.read_csv(params.trainval_csv, index_col=0, parse_dates=True)
df_trainval

,dist_low_30,in_sess_time,dist_low_60,dist_low_28,eng_ema_cross_up,minute_time,rsi,hour_time,z_vwap_dev,dist_high_30,...,bb_w_z_60,minus_di_30,volume_z_30,macd_diff_z_90,plus_di_60,close_raw,signal,pred_signal,ask,bid
2009-01-02 13:30:00,0.169658,0.0,0.121953,0.176068,0.0,0.247492,0.432363,0.2,0.399296,0.197965,...,0.502551,0.475728,1.000000,0.534340,0.231259,3.058929,0.047600,0.103507,3.059387,3.058470
2009-01-02 13:31:00,0.084978,0.0,0.061083,0.088189,0.0,0.249164,0.369266,0.2,0.314610,0.283303,...,0.502850,0.424305,1.000000,0.487908,0.212478,3.053571,0.049326,0.089523,3.054029,3.053113
2009-01-02 13:32:00,0.000000,0.0,0.000000,0.000000,0.0,0.250836,0.319113,0.2,0.230899,0.368941,...,0.502797,0.386817,1.000000,0.431798,0.198286,3.048214,0.051802,0.081353,3.048672,3.047757
2009-01-02 13:33:00,0.086863,0.0,0.062438,0.090145,0.0,0.252508,0.407674,0.2,0.414635,0.238910,...,0.503754,0.346974,1.000000,0.443498,0.198742,3.053690,0.055660,0.086559,3.054149,3.053232
2009-01-02 13:34:00,0.173415,0.0,0.124653,0.179967,0.0,0.254181,0.480449,0.2,0.609879,0.114999,...,0.502366,0.321783,0.904292,0.485573,0.203127,3.059167,0.060387,0.108890,3.059626,3.058708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 20:56:00,0.183189,1.0,0.131679,0.190111,1.0,0.993311,0.583460,0.9,0.317730,0.044131,...,0.539532,0.172834,0.374268,0.530514,0.216956,131.730000,0.036463,0.114993,131.749760,131.710240
2022-12-23 20:57:00,0.210647,1.0,0.151416,0.218606,1.0,0.994983,0.616177,0.9,0.468530,0.016540,...,0.531617,0.171265,0.444313,0.527930,0.213317,131.805000,0.033832,0.127890,131.824771,131.785229
2022-12-23 20:58:00,0.219793,1.0,0.157990,0.228097,1.0,0.996655,0.626702,0.9,0.484463,0.007350,...,0.520810,0.167707,0.415307,0.525205,0.214583,131.830000,0.028223,0.127704,131.849775,131.810226
2022-12-23 20:59:00,0.230763,1.0,0.165876,0.239482,1.0,0.998328,0.639478,0.9,0.453982,0.000000,...,0.510832,0.161685,0.665962,0.523102,0.216984,131.860000,0.024628,0.132516,131.879779,131.840221


In [10]:
# Pre-compute per-day slices once
groups = list(df_trainval.groupby(df_trainval.index.normalize()))

def objective(trial: optuna.Trial) -> float:

    print('Processing hyperparameters, generate actions on the smoothed predicted signal, and simulate trading......')
    
    sign_smoothwin = trial.suggest_categorical("sign_smoothwin", [1,2,3,5,7,10,15])
    sellmin_idx    = trial.suggest_categorical("sellmin_idx", [None, -1, -2, -3, -5, -10, -30])
    buy_thresh     = trial.suggest_float("buy_thresh", 0.05, 0.95)
    trailstop_pct  = trial.suggest_float("trailstop_pct", 0.1, 10)
    
    daily_pnls: List[float] = []

    print('')
    for step, (day, df_day) in enumerate(
        tqdm(groups, desc=f"Trial {trial.number}", leave=False), 1
    ):
        # Causal smoothing of the raw pred_signal
        df_proc = df_day.copy()
        df_proc["pred_signal_smooth"] = (
            df_proc["pred_signal"]
              .rolling(window=sign_smoothwin, min_periods=1)
              .mean()
        )

        # Generate actions on the smoothed column
        df_actions = trades.generate_trade_actions(
            df                = df_proc,
            col_signal        = "pred_signal_smooth",
            col_action        = "pred_action",
            col_price         = "close_raw",
            buy_thresh        = buy_thresh,
            trailstop_pct     = trailstop_pct,
            sellmin_idx       = sellmin_idx
            )

        # Simulate 1-day P&L
        sim = trades.simulate_trading(
            day               = day,
            df                = df_actions,
            col_action        = "pred_action",
            sellmin_idx       = sellmin_idx
        )
   
        _, _, stats = next(iter(sim.values()))
        last_line = stats["STRATEGY"].strip().splitlines()[-1]
        strategy_val = float(last_line.partition("=")[2].strip() or "0.0")
        daily_pnls.append(strategy_val)

        # Explicitly delete large locals at the end of each trial
        del df_proc, df_actions, sim, stats
        gc.collect()

        # Rreport for pruning
        trial.report(strategy_val, int(step))
        if trial.should_prune(): raise optuna.TrialPruned()

    # Average daily P&L
    return float(np.mean(daily_pnls))


In [ ]:
n_trials = 900
n_jobs = 1

pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=500)

study = optuna.create_study( # Point it at an SQLite file so it writes out each result immediately instead of buffering in RAM
    storage=f"sqlite:///{os.path.join(params.optuna_folder, "optuna_study_predicted.db")}", 
    load_if_exists=True,
    direction="maximize",
    pruner=pruner,
)

study.optimize(
    objective,
    n_trials          = n_trials,
    n_jobs            = n_jobs,
    callbacks=[plots.cleanup_callback, plots.lightweight_plot_callback, plots.save_results_callback],
    gc_after_trial    = True,
)

plt.close('all')   # safe here; the final image remains displayed in the notebook output
gc.collect()       # optional extra sweep

[I 2025-11-26 22:53:54,177] A new study created in RDB with name: no-name-e4795acf-5be1-4a71-abe1-23fc8080e901


Processing hyperparameters, generate actions on the smoothed predicted signal, and simulate trading......



Trial 0:   0%|          | 0/3520 [00:00<?, ?it/s]

In [ ]:
# Final plots & JSON dump 
ax = plot_optimization_history(study)
ax.figure.set_size_inches(8, 4)
plt.show()

print("Best Parameters       :", study.best_params)
print("Best Average Daily P&L:", study.best_value)

importances = get_param_importances(study)
print("\nHyperparameter importances (higher ⇒ more impact):")
for name, score in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"  {name:20s}: {score:.3f}")

first_day = df_trainval.index.normalize().min().strftime("%Y%m%d")
last_day  = df_trainval.index.normalize().max().strftime("%Y%m%d")
file_name = f"{params.ticker}_{first_day}-{last_day}_optuna_predicted_{study.best_value}_{params.model_path}.json"
file_path = os.path.join(params.optuna_folder, file_name)

with open(file_path, "w") as f:
    json.dump({
        "best_params": study.best_params,
        "best_value" : study.best_value,
        "importances": importances,
        "trials": [
            {"number": t.number, "value": t.value, "params": t.params, "state": t.state.name}
            for t in study.trials
        ],
    }, f, indent=4)

print(f"\nOptuna results (and importances) saved to: {file_path}")